In [1]:
import pandas as pd
import numpy as np
import random
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
feat_thresh_arr = [{'name':'hemoglobin', 'thresh':[12], 'std':2},
                   {'name':'ferritin', 'thresh':[30, 100], 'std':2},
                   {'name':'ret_count', 'thresh':[2], 'std':0.2},
                   {'name':'segmented_neutrophils', 'thresh':[0], 'std':2},
                   {'name':'tibc', 'thresh':[450], 'std':50},
                   {'name':'mcv', 'thresh':[80,100], 'std':2}]

anemias_features_dict = {'Vitamin B12/Folate deficiency anemia': ['mcv', 'segmented_neutrophils'], 
                         'Unspecified anemia': ['mcv', 'segmented_neutrophils'], 
                         'Anemia of chronic disease': ['mcv', 'ferritin', 'tibc'], 
                         'Iron deficiency anemia': ['mcv', 'ferritin', 'tibc'], 
                         'Hemolytic anemia': ['mcv', 'ret_count'], 
                         'Aplastic anemia': ['mcv', 'ret_count']}

#### The data

In [3]:
all_df= pd.read_csv('../../data/more_features/more_feats_correlated_0.1.csv')
all_df = all_df.fillna(-1)
all_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.183192,187.573466,4.951674,1.661383,316.593436,95.006424,181.242992,3.531295,1,-1.000000,28.040619,96.721542,49.530542,23.186628,-1.000000,33.549575,57.247868,Hemolytic anemia
1,14.387445,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,125.249617,-1.000000,1,-1.000000,98.357508,112.758764,62.464566,7.168920,-1.000000,43.162335,-1.000000,No anemia
2,12.749357,5.012158,3.502800,6.179371,498.418768,76.759285,159.834784,4.982859,1,1.022939,56.850479,75.739552,72.072041,20.600875,44.872138,38.248071,32.068372,Iron deficiency anemia
3,11.508870,197.180945,1.200125,0.000000,457.033309,102.900301,131.177927,3.355346,1,-1.000000,111.220307,66.999185,18.353272,14.132423,-1.000000,34.526609,28.702050,Unspecified anemia
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.000000,2.713693,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,28.369968,-1.000000,Vitamin B12/Folate deficiency anemia


In [4]:
len(all_df)

70000

In [5]:
class_dict = constants.CLASS_DICT
#print(all_df.label.unique())
all_df['label'] = all_df['label'].replace(class_dict)
X = all_df.iloc[:, 0:-1]
y = all_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train_df = pd.concat([X_train, y_train], axis=1)
X_test_df = pd.concat([X_test, y_test], axis=1)
X_test_df = X_test_df.reset_index(drop=True)
X_test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.391136,-1.000000,1.504298,5.058881,467.197112,96.252926,217.860499,3.550376,1,-1.000000,36.703810,122.310168,49.897027,20.353251,-1.000000,34.173407,46.631388,6
1,10.355048,272.506966,-1.000000,0.000000,128.706486,101.912313,-1.000000,3.048223,1,-1.000000,-1.000000,-1.000000,29.327349,-1.000000,-1.000000,31.065144,-1.000000,2
2,11.159385,-1.000000,-1.000000,0.000000,-1.000000,103.395616,77.212369,3.237870,0,-1.000000,8.690122,65.554731,19.167967,7.239049,-1.000000,33.478155,-1.000000,2
3,8.179735,140.876632,5.484515,-1.000000,303.740826,99.877458,155.573175,2.456931,0,-1.000000,41.123526,119.433840,39.384848,6.394235,-1.000000,24.539204,51.219053,5
4,9.916825,-1.000000,3.909810,-1.000000,-1.000000,90.543986,189.347916,3.285748,0,1.501433,34.905667,59.177001,58.538524,22.843594,139.245204,29.750475,-1.000000,5


In [25]:
X_train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
21569,7.559457,227.891116,5.805018,2.878656,147.846935,103.948918,106.431924,2.181684,1,-1.000000,2.986261,114.459461,13.468107,15.321222,-1.00000,22.678370,71.987914,1
50615,12.151687,49.933676,4.348977,0.731308,206.283448,88.205350,66.458334,4.132976,0,-1.000000,72.488033,85.729484,20.881910,11.104061,-1.00000,36.455062,32.216998,0
63205,7.684368,244.551542,4.040941,0.000000,-1.000000,103.179020,-1.000000,2.234282,0,0.683808,-1.000000,-1.000000,77.068526,-1.000000,112.25213,23.053104,-1.000000,2
42257,6.783503,290.314245,1.180064,0.000000,184.535734,103.283728,181.424498,1.970350,1,1.556780,74.325537,95.770863,45.787469,1.605937,-1.00000,20.350509,98.314019,2
25700,6.162946,25.460454,2.241183,0.000000,-1.000000,104.151962,246.116674,1.775179,1,0.842525,49.891356,124.972794,24.090046,18.002089,-1.00000,18.488839,-1.000000,2


In [6]:
# X_test_df.to_csv('../../data/more_features/train_sets/test_set_constant.csv', index=False)

In [7]:
def get_dict(dict_arr, feat_name):
    for dictionary in dict_arr:
        if dictionary['name']==feat_name:
            return dictionary

In [8]:
def feat_noisiness(df, feat_name, frac, mean, std):
    df[feat_name] = df[feat_name].sample(frac=1-frac)
    if feat_name ==  'segmented_neutrophils': #how was this a good idea
        df[feat_name] = df[feat_name].fillna(-1)
    else:
        nan_index = df[df[feat_name].isna()].index
        fill_values = np.random.normal(mean, std, size=len(nan_index))
        df[feat_name] = df[feat_name].fillna(pd.Series(fill_values, index=nan_index))
    return df[feat_name]

In [9]:
total_frac = 0.5

In [10]:
noisy_anemia_dfs_list = []
for label in X_train_df.label.unique():
    anemia = constants.ACTION_SPACE[label]
    print(f'{label} - {constants.ACTION_SPACE[label]}')
    if anemia not in ['No anemia', 'Inconclusive diagnosis']:
        anemia_df = X_train_df[X_train_df.label==label]
        feature_num = len(anemias_features_dict[anemia])
        frac = total_frac/feature_num
        for feat in anemias_features_dict[anemia]:
            feat_info = get_dict(feat_thresh_arr, feat)
            threshes, std = feat_info['thresh'], feat_info['std']
            print(f'feat:{feat}, threshes:{threshes}, std:{std}')
            for thresh in threshes:
                anemia_df[feat] = feat_noisiness(anemia_df, feat, frac/len(threshes), thresh, std)
        noisy_anemia_dfs_list.append(anemia_df)

1 - Vitamin B12/Folate deficiency anemia
feat:mcv, threshes:[80, 100], std:2
feat:segmented_neutrophils, threshes:[0], std:2
0 - No anemia
2 - Unspecified anemia
feat:mcv, threshes:[80, 100], std:2
feat:segmented_neutrophils, threshes:[0], std:2
3 - Anemia of chronic disease
feat:mcv, threshes:[80, 100], std:2
feat:ferritin, threshes:[30, 100], std:2
feat:tibc, threshes:[450], std:50
5 - Hemolytic anemia
feat:mcv, threshes:[80, 100], std:2
feat:ret_count, threshes:[2], std:0.2
4 - Iron deficiency anemia
feat:mcv, threshes:[80, 100], std:2
feat:ferritin, threshes:[30, 100], std:2
feat:tibc, threshes:[450], std:50
6 - Aplastic anemia
feat:mcv, threshes:[80, 100], std:2
feat:ret_count, threshes:[2], std:0.2
7 - Inconclusive diagnosis


In [11]:
noisy_anemia_dfs_list[0].describe()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.0
mean,9.485649,178.908222,0.810852,2.420083,108.017326,99.810409,84.691100,2.776392,0.565278,-0.047020,47.251847,50.677318,28.605000,9.236849,23.525831,28.456948,11.566602,1.0
std,2.010686,165.020410,2.310765,2.617574,160.943496,7.055913,83.897310,0.589632,0.495755,1.104016,50.294430,45.606453,26.750876,10.326430,43.046138,6.032059,29.666639,0.0
min,6.000016,-1.000000,-1.000000,-1.000000,-1.000000,72.582407,-1.000000,1.727967,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,18.000047,-1.000000,1.0
25%,7.733065,-1.000000,-1.000000,-0.174905,-1.000000,100.525705,-1.000000,2.259337,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,23.199196,-1.000000,1.0
50%,9.484115,153.866410,-1.000000,2.419525,-1.000000,101.888604,68.405890,2.771165,1.000000,-1.000000,31.992806,51.311643,24.748857,6.612607,-1.000000,28.452344,-1.000000,1.0
75%,11.246380,323.286966,2.733409,4.703520,225.409929,103.369910,158.110979,3.289616,1.000000,1.017853,90.898066,91.652273,51.706486,18.211353,47.190413,33.739139,-1.000000,1.0
max,12.998632,499.921689,5.995977,6.998971,499.598278,105.569176,249.919719,3.887936,1.000000,1.998723,149.997358,129.985350,79.958518,29.990223,139.777727,38.995895,233.686429,1.0


In [12]:
no_df = X_train_df[X_train_df.label==constants.CLASS_DICT['No anemia']]
inconc_df = X_train_df[X_train_df.label==constants.CLASS_DICT['Inconclusive diagnosis']]
len(no_df), len(inconc_df)

(8000, 3958)

In [13]:
noisy_anemia_dfs_list.append(no_df)
noisy_anemia_dfs_list.append(inconc_df)
len(noisy_anemia_dfs_list)

8

In [14]:
combined_df = pd.concat(noisy_anemia_dfs_list, axis=0)
combined_df = combined_df.sample(frac=1).reset_index(drop=True)
combined_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.733539,98.760455,-1.000000,6.276499,496.885050,92.391638,200.002154,2.511117,0,0.766438,107.065553,60.803822,10.327995,2.049664,-1.0,23.200617,40.251192,7
1,6.571819,-1.000000,1.732890,5.595521,349.256539,99.437222,-1.000000,1.982704,1,-1.000000,-1.000000,-1.000000,63.259799,-1.000000,-1.0,19.715457,-1.000000,5
2,9.263902,59.085148,1.797073,0.801931,399.726111,102.617378,-1.000000,2.708285,1,-1.000000,-1.000000,-1.000000,35.768476,-1.000000,-1.0,27.791707,-1.000000,1
3,6.205564,155.204031,0.191721,0.000000,-1.000000,101.490214,202.012762,1.834334,0,-1.000000,86.144828,69.611146,-1.000000,25.927446,-1.0,18.616693,-1.000000,2
4,7.807171,-1.000000,-1.000000,0.000000,-1.000000,100.914488,-1.000000,2.320927,0,-1.000000,-1.000000,-1.000000,60.558098,-1.000000,-1.0,23.421513,-1.000000,2


#### The correlated features

In [15]:
def get_rbc(row):
    if row['mcv'] == -1:
        return -1
    else:
        return (30*row['hemoglobin'])/row['mcv']

def get_tsat(row):
    if (row['serum_iron']==-1) | (row['tibc']==-1):
        return -1
    else:
        return (row['serum_iron']/row['tibc'])*100

In [16]:
combined_df['rbc'] = combined_df.apply(lambda row: get_rbc(row), axis=1)
combined_df['tsat'] = combined_df.apply(lambda row: get_tsat(row), axis=1)
combined_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.733539,98.760455,-1.000000,6.276499,496.885050,92.391638,200.002154,2.511117,0,0.766438,107.065553,60.803822,10.327995,2.049664,-1.0,23.200617,40.251192,7
1,6.571819,-1.000000,1.732890,5.595521,349.256539,99.437222,-1.000000,1.982704,1,-1.000000,-1.000000,-1.000000,63.259799,-1.000000,-1.0,19.715457,-1.000000,5
2,9.263902,59.085148,1.797073,0.801931,399.726111,102.617378,-1.000000,2.708285,1,-1.000000,-1.000000,-1.000000,35.768476,-1.000000,-1.0,27.791707,-1.000000,1
3,6.205564,155.204031,0.191721,0.000000,-1.000000,101.490214,202.012762,1.834334,0,-1.000000,86.144828,69.611146,-1.000000,25.927446,-1.0,18.616693,-1.000000,2
4,7.807171,-1.000000,-1.000000,0.000000,-1.000000,100.914488,-1.000000,2.320927,0,-1.000000,-1.000000,-1.000000,60.558098,-1.000000,-1.0,23.421513,-1.000000,2


#### Randomly modifying to no anemia

In [17]:
anemic_df = combined_df[combined_df.label != constants.CLASS_DICT['No anemia']]
non_anemic_df = combined_df[combined_df.label == constants.CLASS_DICT['No anemia']]
len(anemic_df), len(non_anemic_df)

(48000, 8000)

In [18]:
anemic_df.label.value_counts()

3    7805
4    7414
2    7226
6    7216
1    7200
5    7181
7    3958
Name: label, dtype: int64

In [19]:
anemic_df['label'] = anemic_df['label'].sample(frac=1-0.1)
anemic_df['label'] = anemic_df['label'].fillna(constants.CLASS_DICT['No anemia'])
anemic_df.label.value_counts()

3.0    6992
4.0    6639
2.0    6534
6.0    6530
1.0    6477
5.0    6459
0.0    4800
7.0    3569
Name: label, dtype: int64

#### Finalizing 

In [20]:
final_df = pd.concat([non_anemic_df, anemic_df], axis=0)
final_df = final_df.sample(frac=1).reset_index(drop=True)
final_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,10.693201,266.041081,-1.000000,0.000000,-1.000000,100.139498,99.276333,3.203491,0,-1.000000,109.220956,52.958486,47.800548,27.035521,73.231870,32.079602,-1.000000,2.0
1,15.510675,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,167.244117,-1.000000,0,0.947794,19.395071,62.854309,32.406311,8.722673,113.516226,46.532026,-1.000000,0.0
2,12.874539,70.601935,-1.000000,1.167703,213.148487,98.726520,91.897116,3.912183,1,-1.000000,97.245733,129.734764,50.229998,13.014397,-1.000000,38.623618,43.114130,3.0
3,8.490920,488.313393,3.119245,0.233657,294.334880,97.470161,158.941654,2.613390,0,0.337663,63.356317,116.003109,57.132707,5.582379,-1.000000,25.472759,54.000278,5.0
4,16.339183,-1.000000,1.057889,3.671608,120.286086,-1.000000,99.970392,-1.000000,1,0.979521,53.380678,99.123833,51.802845,28.098360,70.583308,49.017549,83.110521,0.0


In [21]:
final_df.label.value_counts()

0.0    12800
3.0     6992
4.0     6639
2.0     6534
6.0     6530
1.0     6477
5.0     6459
7.0     3569
Name: label, dtype: int64

In [22]:
utils.get_dt_performance(final_df, 'numeric')

(0.6775892857142857,
 0.6761966102367933,
 0.8134008743521025,
 datetime.timedelta(microseconds=1984))

In [23]:
final_df.to_csv('../../data/more_features/train_sets/train_set_noisy_5.csv', index=False)

In [24]:
len(final_df)

56000